In [60]:
# """Module for data-related stuff."""



# class DrivingDataset:

#     CLASSES = {
#         "speedbumppassing": "speedbumppassing",
#         "zigzag": "zigzag",
#     }

#     FEATURES = [
#         "ax",
#         "ay",
#         "az",
#         "wx",
#         "wy",
#         "wz"
#     ]

#     def __init__(self, data_root, unwrapped_attitude=False,
#                  metadata_file=None):
#         self.data_root = pathlib.Path(data_root)
#         self.files = []
#         self.unwrapped_attitude = unwrapped_attitude
#         self.metadata = {}  # Dictionary with participant codes as keys.

#         # Save each CSV file and infer class from filename.
#         for csv_ in self.data_root.glob("**/*.csv"):
#             class_ = str(csv_.parent.stem)[:3]
#             if class_ in self.CLASSES.keys():
#                 self.files.append([csv_, class_])

#         # Read metadata form given file.
#         if metadata_file:
#             with open(metadata_file, newline="") as metadata:
#                 csv_reader = csv.reader(metadata)
#                 next(csv_reader, None)  # skip the headers
#                 for row in csv_reader:
#                     self.metadata[row[0]] = list(map(int, row[1:]))

#     def __getitem__(self, item):
#         file_, class_ = self.files[item]
#         signals = csv2numpy(file_)

#         if self.unwrapped_attitude:
#             # Unwrap attitude signals.
#             for i in range(3):
#                 signals[:, i] = np.unwrap(signals[:, i])

#         if self.metadata:
#             # Read metadata and return as extra element.
#             metadata = self.metadata[file_.stem.split("_")[1]]
#             return signals, class_, metadata
#         return signals, class_

#     def __len__(self):
#         return len(self.files)


# class HARDatasetCrops(HARDataset):
#     """Dataset with fixed-length crops.

#     Args:
#         data_root -- string. Path to data directory.
#         length -- int. Crops length.
#         discard_start -- int. Number of samples to discard from start.
#         discard_end -- int. Number of samples to discard from end.
#         unwrapped_attitude -- bool. Whether to unwrap attitude signals.
#         padding_mode -- None or string. If None, the samples not fitting in
#                 integer number of windows will be discarded. If string,
#                 the value will be passed to numpy's pad function.
#     """

#     def __init__(self, data_root, length, discard_start, discard_end,
#                  unwrapped_attitude=True, padding_mode=None,
#                  metadata_file=None):
#         super().__init__(data_root, unwrapped_attitude=unwrapped_attitude,
#                          metadata_file=metadata_file)
#         self.length = length
#         self.discard_start = discard_start
#         self.discard_end = discard_end
#         self.padding_mode = padding_mode

#         self.crops = self.get_crops()

#     def get_crops(self):
#         """Return list with crops from files."""
#         crops = []
#         # Iterate over data files.
#         for file, class_ in self.files:
#             # Read from file.
#             signal = csv2numpy(file)
#             # Crop start and end.
#             signal = signal[self.discard_start:(signal.shape[0] - self.discard_end)]
#             windows, remainder = divmod(signal.shape[0], self.length)
#             if self.padding_mode and remainder != 0:
#                 # Apply padding with given padding mode.
#                 padding = self.length * (windows + 1) - signal.shape[0]
#                 signal = np.pad(signal, ((0, padding), (0, 0)), self.padding_mode)
#             elif self.padding_mode is None:
#                 # Crop the end.
#                 signal = signal[:(self.length * windows)]
#             # Obtain crops from <discard_start> to <discard-end>.
#             for i in range(0, signal.shape[0], self.length):
#                 crop = signal[i:(i + self.length)]
#                 if self.unwrapped_attitude:
#                     # Unwrap phase of first 3 features (attitude signals).
#                     for s in range(3):
#                         crop[:, s] = np.unwrap(crop[:, s])
#                 if self.metadata:
#                     # Read metadata and return as extra element.
#                     metadata = self.metadata[file.stem.split("_")[1]]
#                     crops.append([crop, class_, metadata])
#                 else:
#                     crops.append([crop, class_])

#         return crops

#     def __getitem__(self, item):
#         return self.crops[item]

#     def __len__(self):
#         return len(self.crops)


# if __name__ == '__main__':
#     dataset = HARDatasetCrops('motionsense-dataset', 256, 10, 10, True)
#     for item in iter(dataset):
#         assert item[0].shape == (256, 12)

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import keras
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, concatenate, Input

In [2]:
import os
files_train = []
for i in os.listdir("zigzag/train"):
    files_train.append(["zigzag/train/"+i, "zigzag"])
print(len(files_train))    
    
for i in os.listdir("speedbumppassing/train"):
    files_train.append(["speedbumppassing/train/"+i, "speedbump"])
    
for i in os.listdir("potholes/train"):
    files_train.append(["potholes/train/"+i, "pothole"])

70


In [3]:
import os
files_test = []
for i in os.listdir("zigzag/test"):
    files_test.append(["zigzag/test/"+i, "zigzag"])
print(len(files_test))    
    
for i in os.listdir("speedbumppassing/test"):
    files_test.append(["speedbumppassing/test/"+i, "speedbump"])
    
for i in os.listdir("potholes/test"):
    files_test.append(["potholes/test/"+i, "pothole"])

5


In [4]:
import pathlib
import csv

import numpy as np


def csv2numpy(file_name):
    """Read multidimensional signal from file"""
    # Read data from file.
    data = np.genfromtxt(file_name, delimiter=",", skip_header=1)
    # Return all columns but the first one (as it is the index).
    return data[:, 1:-1]



def get_crops(files, length, discard_start, discard_end, padding_mode=None):
        """Return list with crops from files."""
        crops = []
        # Iterate over data files.
        for file, class_ in files:
            # Read from file.
            signal = csv2numpy(file)
            # Crop start and end.
            signal = signal[discard_start:(signal.shape[0] - discard_end)]
            windows, remainder = divmod(signal.shape[0], length)
            if padding_mode and remainder != 0:
                # Apply padding with given padding mode.
                padding = length * (windows + 1) - signal.shape[0]
                signal = np.pad(signal, ((0, padding), (0, 0)), padding_mode)
            elif padding_mode is None:
                # Crop the end.
                signal = signal[:(length * windows)]
            # Obtain crops from <discard_start> to <discard-end>.
            for i in range(0, signal.shape[0], length):
                crop = signal[i:(i + length)]
                crops.append([crop, class_])

        return crops

In [5]:
crops_train = get_crops(files_train, 245, 50, 50)
crops_test = get_crops(files_test, 245, 50, 50)

In [6]:
label_encoder = sklearn.preprocessing.LabelEncoder()
label_encoder.fit(["zigzag", "speedbump", "pothole"]);

In [7]:
X_train = []
y_train = []
for i in crops_train:
    X_train.append(i[0])
    y_train.append(i[1])


In [8]:
zigzag_count = y_train.count("zigzag")
speedbump_count = y_train.count("speedbump")
pothole_count = y_train.count("pothole")

print("zigzag count:", zigzag_count)
print("speedbump count:", speedbump_count)
print("pothole count:", pothole_count)

zigzag count: 15253
speedbump count: 685
pothole count: 680


In [9]:
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Mapping:", mapping)

Mapping: {'pothole': 0, 'speedbump': 1, 'zigzag': 2}


In [10]:
X_test = []
y_test = []
for i in crops_test:
    X_test.append(i[0])
    y_test.append(i[1])

In [11]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [12]:
y_train = to_categorical(label_encoder.transform(y_train))
y_test = to_categorical(label_encoder.transform(y_test))


In [13]:
X_train.shape

(16618, 245, 6)

In [14]:
X_test.shape

(817, 245, 6)

In [15]:
import numpy as np

pothole = np.array([1., 0., 0.])
speedbump = np.array([0., 1., 0.])
zigzag = np.array([0., 0., 1.])

zigzagc = np.sum(np.all(y_train == zigzag, axis=1))
speedbumpc = np.sum(np.all(y_train == speedbump, axis=1))
potholec = np.sum(np.all(y_train == pothole, axis=1))


In [16]:
zigzagc, speedbumpc, potholec

(15253, 685, 680)

In [17]:
import numpy as np

pothole = np.array([1., 0., 0.])
speedbump = np.array([0., 1., 0.])
zigzag = np.array([0., 0., 1.])

zigzagc = np.sum(np.all(y_test == zigzag, axis=1))
speedbumpc = np.sum(np.all(y_test == speedbump, axis=1))
potholec = np.sum(np.all(y_test == pothole, axis=1))


In [20]:
zigzagc, speedbumpc, potholec

(564, 126, 127)

In [18]:
y_train.shape

(16618, 3)

In [19]:
y_test.shape

(817, 3)

In [59]:
unique_values, counts = np.unique(y_test, return_counts=True)

# Display the results
for value, count in zip(unique_values, counts):
    print(f"Value {value}: Count {count}")

Value 0.0: Count 1634
Value 1.0: Count 817


# Simple CNN

In [153]:
clf = Sequential()
#add model layers
clf.add(Conv1D(16, kernel_size=5, activation="relu", input_shape=(245, 6)))
clf.add(Conv1D(32, kernel_size=5, activation="relu"))

clf.add(Flatten())
clf.add(Dense(16, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)))
clf.add(Dense(3, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01)))

In [154]:
clf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

clf.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

Epoch 1/30
63/63 [==============================] - 3s 26ms/step - loss: 0.7032 - accuracy: 0.8589 - val_loss: 3.7193 - val_accuracy: 0.3158
Epoch 2/30
63/63 [==============================] - 0s 6ms/step - loss: 0.3623 - accuracy: 0.9501 - val_loss: 4.7844 - val_accuracy: 0.2803
Epoch 3/30
63/63 [==============================] - 0s 7ms/step - loss: 0.2711 - accuracy: 0.9616 - val_loss: 3.8628 - val_accuracy: 0.3721
Epoch 4/30
63/63 [==============================] - 0s 6ms/step - loss: 0.2178 - accuracy: 0.9721 - val_loss: 5.3319 - val_accuracy: 0.3341
Epoch 5/30
63/63 [==============================] - 0s 7ms/step - loss: 0.2183 - accuracy: 0.9651 - val_loss: 4.8376 - val_accuracy: 0.3647
Epoch 6/30
63/63 [==============================] - 0s 7ms/step - loss: 0.1613 - accuracy: 0.9816 - val_loss: 4.2661 - val_accuracy: 0.3929
Epoch 7/30
63/63 [==============================] - 0s 7ms/step - loss: 0.1281 - accuracy: 0.9910 - val_loss: 4.2035 - val_accuracy: 0.4186
Epoch 8/30
63/63 [=

In [42]:
clf.evaluate(X_test, y_test)

26/26 [==============================] - 0s 3ms/step - loss: 1.7504 - accuracy: 0.8690


[1.7504258155822754, 0.8690330386161804]

In [55]:
clf2 = Sequential()
#add model layers
clf2.add(Conv1D(16, kernel_size=5, activation="relu", input_shape=(245, 6)))
clf2.add(Conv1D(32, kernel_size=5, activation="relu"))

clf2.add(Flatten())
clf2.add(Dense(8, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)))
clf2.add(Dense(3, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01)))

In [56]:
clf2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

clf2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

Epoch 1/30
520/520 [==============================] - 4s 6ms/step - loss: 0.3058 - accuracy: 0.9295 - val_loss: 0.9730 - val_accuracy: 0.8348
Epoch 2/30
520/520 [==============================] - 3s 6ms/step - loss: 0.1436 - accuracy: 0.9593 - val_loss: 1.1086 - val_accuracy: 0.8507
Epoch 3/30
520/520 [==============================] - 3s 6ms/step - loss: 0.1062 - accuracy: 0.9792 - val_loss: 1.2036 - val_accuracy: 0.8421
Epoch 4/30
520/520 [==============================] - 3s 5ms/step - loss: 0.0827 - accuracy: 0.9844 - val_loss: 1.3781 - val_accuracy: 0.8311
Epoch 5/30
520/520 [==============================] - 3s 5ms/step - loss: 0.0686 - accuracy: 0.9883 - val_loss: 1.6799 - val_accuracy: 0.8482
Epoch 6/30
520/520 [==============================] - 3s 6ms/step - loss: 0.0553 - accuracy: 0.9917 - val_loss: 1.3522 - val_accuracy: 0.8690
Epoch 7/30
520/520 [==============================] - 3s 6ms/step - loss: 0.0645 - accuracy: 0.9895 - val_loss: 1.4544 - val_accuracy: 0.8568
Epoch 

In [57]:
clf2.evaluate(X_test, y_test)

26/26 [==============================] - 0s 2ms/step - loss: 1.0705 - accuracy: 0.8715


[1.0704624652862549, 0.8714810013771057]

# Transformer

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Imports
import pandas as pd
import numpy as np
from keras import layers


#Splitting data
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


#Building Transformer
import torch
import torch.nn as nn 
import torch.optim as optim
import math


In [16]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [17]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [18]:
input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=2,
    num_heads=4,
    ff_dim=3,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.25,
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_data = (X_test, y_test),
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
)



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 245, 6)]             0         []                            
                                                                                                  
 multi_head_attention (Mult  (None, 245, 6)               222       ['input_1[0][0]',             
 iHeadAttention)                                                     'input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 245, 6)               0         ['multi_head_attention[0][0]']
                                                                                                  
 layer_normalization (Layer  (None, 245, 6)               12        ['dropout[0][0]']         

 tf.__operators__.add_5 (TF  (None, 245, 6)               0         ['layer_normalization_5[0][0]'
 OpLambda)                                                          , 'tf.__operators__.add_4[0][0
                                                                    ]']                           
                                                                                                  
 multi_head_attention_3 (Mu  (None, 245, 6)               222       ['tf.__operators__.add_5[0][0]
 ltiHeadAttention)                                                  ',                            
                                                                     'tf.__operators__.add_5[0][0]
                                                                    ']                            
                                                                                                  
 dropout_6 (Dropout)         (None, 245, 6)               0         ['multi_head_attention_3[0][0]
          

In [19]:
model.evaluate(X_test, y_test)

26/26 [==============================] - 4s 143ms/step - loss: 0.9925 - categorical_accuracy: 0.7099


[0.9925313591957092, 0.7099143266677856]

# CNN-LSTM

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, LSTM, Dense
from tensorflow.keras.regularizers import l2

# Define the model with L2 regularization
model = Sequential()

# TimeDistributed(Conv1D()) layer with parameters and L2 regularization
model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)), input_shape=(245, 6, 1)))

# TimeDistributed(MaxPooling1D()) layer with parameters and padding
model.add(TimeDistributed(MaxPooling1D(pool_size=2, padding='same')))

# TimeDistributed(Conv1D()) layer with parameters and L2 regularization
model.add(TimeDistributed(Conv1D(filters=8, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.01))))

# TimeDistributed(MaxPooling1D()) layer with parameters and padding
model.add(TimeDistributed(MaxPooling1D(pool_size=2, padding='same')))

# TimeDistributed(Conv1D()) layer with parameters and L2 regularization
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.01))))

# TimeDistributed(MaxPooling1D()) layer with parameters and padding
model.add(TimeDistributed(MaxPooling1D(pool_size=2, padding='same')))

# TimeDistributed(Conv1D()) layer with parameters and L2 regularization
model.add(TimeDistributed(Conv1D(filters=8, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(0.01))))

# TimeDistributed(MaxPooling1D()) layer with parameters and padding
model.add(TimeDistributed(MaxPooling1D(pool_size=2, padding='same')))

# TimeDistributed(Flatten()) layer
model.add(TimeDistributed(Flatten()))

# Define LSTM model with parameters and L2 regularization
model.add(LSTM(units=60, return_sequences=True, kernel_regularizer=l2(0.01)))

# Additional LSTM layer with L2 regularization
model.add(LSTM(units=70, return_sequences=True, kernel_regularizer=l2(0.01)))

# Flatten layer outside TimeDistributed
model.add(Flatten())

# Dense layer with parameters and L2 regularization
model.add(Dense(units=3, activation='softmax', kernel_regularizer=l2(0.01)))  # Adjust units to match the number of output classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
520/520 [==============================] - 128s 233ms/step - loss: 0.5720 - accuracy: 0.9162 - val_loss: 0.9867 - val_accuracy: 0.6903
Epoch 2/30
520/520 [==============================] - 121s 232ms/step - loss: 0.3647 - accuracy: 0.9171 - val_loss: 1.2231 - val_accuracy: 0.6891
Epoch 3/30
520/520 [==============================] - 119s 229ms/step - loss: 0.3543 - accuracy: 0.9171 - val_loss: 0.9693 - val_accuracy: 0.6903
Epoch 4/30
520/520 [==============================] - 120s 232ms/step - loss: 0.3241 - accuracy: 0.9242 - val_loss: 0.6528 - val_accuracy: 0.8360
Epoch 5/30
128/520 [======>.......................] - ETA: 1:32 - loss: 0.2821 - accuracy: 0.9363

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

# TCN

In [65]:
!pip install keras-tcn

  Obtaining dependency information for tensorflow-addons from https://files.pythonhosted.org/packages/ec/52/047d768c4669db0c059109a88c21a3c71bcda957c46f13967e44b8c7fa4c/tensorflow_addons-0.22.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/719.8 kB ? eta -:--:--
   - ------------------------------------- 30.7/719.8 kB 660.6 kB/s eta 0:00:02
   ----- ---------------------------------- 92.2/719.8 kB 1.1 MB/s eta 0:00:01
   ----------- ---------------------------- 204.8/719.8 kB 1.6 MB/s eta 0:00:01
   ----------------- ---------------------- 307.2/719.8 kB 1.7 MB/s eta 0:00:01
   ------------------------ --------------- 440.3/719.8 kB 1.8 MB/s eta 0:00:01
   -------------------------------- ------- 583.7/719.8 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------  716.8/719.8 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 719.8/719.8 kB 2.2 MB/s eta 0:00:00


DEPRECATION: neuralplot 0.0.8 has a non-standard dependency specifier matplotlib>=3.1numpy>=1.16. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of neuralplot or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
# Build TCN Model
from tcn import TCN
def build_tcn_model(input_shape, num_classes):
    model = tf.keras.Sequential()

    # Add TCN layer
    model.add(TCN(input_shape=input_shape, nb_filters=64, kernel_size=3, dilations=[1, 2, 4, 8], return_sequences=False))

    # Fully Connected Layer
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Build the model
input_shape = X_train.shape[1:]
model = build_tcn_model(input_shape, 2)

# Display the model summary
model.summary()

# Train the Model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)




In [ ]:
model.evaluate(X_test, y_test)